!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
pip install --no-cache-dir --index-url https://pypi.nvidia.com pytorch-quantization !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 ./ch transform --config configs/examples/toy_uniform_tensorRT.toml --load /root/mase/TensorRTDev/jsc-tiny_classification_jsc_2024-03-03/software/training_ckpts/best.ckpt --load-type pl
./ch train --config configs/examples/toy_uniform_tensorRT.toml
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

sudo apt update && sudo apt upgrade -y 

wget https://repo.continuum.io/archive/Anaconda3-2023.09-0-Linux-x86_64.sh 

bash Anaconda3-2023.09-0-Linux-x86_64.sh 

Close and reopen terminal

source /root/.bashrc

conda config --set auto_activate_base false

git clone https://github.com/mau-mar/mase/

cd mase

bash scripts/init-conda.sh

source /opt/conda/etc/profile.d/conda.sh

conda activate mase

conda config --add channels conda-forge

git checkout origin/mauro-tensorRT-integration

cuda-python
absl-py
scipy
prettytable
sphinx-glpi-theme

./ch train --config configs/examples/toy_uniform_tensorRT.toml

In [1]:
import sys
import logging
import os
from pathlib import Path
from pprint import pprint as pp

# # figure out the correct path
machop_path = Path(".").resolve().parent /"machop"
assert machop_path.exists(), "Failed to find machop at: {}".format(machop_path)
sys.path.append(str(machop_path))
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch_tensorrt

from torch.utils.tensorboard import SummaryWriter

import pytorch_quantization
from pytorch_quantization import nn as quant_nn
from pytorch_quantization import quant_modules
from pytorch_quantization.tensor_quant import QuantDescriptor
from pytorch_quantization import calib
from tqdm import tqdm

print(pytorch_quantization.__version__)

from chop.dataset import MaseDataModule, get_dataset_info
from chop.tools.logger import set_logging_verbosity

from chop.passes.graph import (
    save_node_meta_param_interface_pass,
    report_node_meta_param_analysis_pass,
    profile_statistics_analysis_pass,
    add_common_metadata_analysis_pass,
    init_metadata_analysis_pass,
    add_software_metadata_analysis_pass,
    tensorrt_quantize_transform_pass,
    tensorrt_calibrate_transform_pass,
    tensorrt_train_transform_pass,
    )
from chop.tools.get_input import InputGenerator
from chop.tools.checkpoint_load import load_model
from chop.ir import MaseGraph

from chop.models import get_model_info, get_model

set_logging_verbosity("info")

/opt/conda/envs/mase/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2.1.3


INFO     Set logging level to info
I0304 19:48:50.862745 139852009707328 logger.py:44] Set logging level to info


In [2]:
model_name = "jsc-tiny"
dataset_name = "jsc"
max_epochs = 1
batch_size = 256
learning_rate = 1e-3
accelerator = "gpu"

data_module = MaseDataModule(
    name=dataset_name,
    batch_size=batch_size,
    model_name=model_name,
    num_workers=0,
)
data_module.prepare_data()
data_module.setup()

In [3]:
# 📝️ change this CHECKPOINT_PATH to the one you trained in Lab1
CHECKPOINT_PATH = "/root/mase/TensorRTDev/jsc-tiny_classification_jsc_2024-03-03/software/training_ckpts/best.ckpt"
model_info = get_model_info(model_name)
# quant_modules.initialize()
model = get_model(
    model_name,
    task="cls",
    dataset_info=data_module.dataset_info,
    pretrained=False)

model = load_model(load_name=CHECKPOINT_PATH, load_type="pl", model=model)

INFO     Loaded pytorch lightning checkpoint from /root/mase/TensorRTDev/jsc-tiny_classification_jsc_2024-03-03/software/training_ckpts/best.ckpt
I0304 19:48:55.801796 139852009707328 checkpoint_load.py:85] Loaded pytorch lightning checkpoint from /root/mase/TensorRTDev/jsc-tiny_classification_jsc_2024-03-03/software/training_ckpts/best.ckpt


In [4]:
mg = MaseGraph(model=model)

In [5]:
import toml

# Path to your TOML file
toml_file_path = '/root/mase/machop/configs/examples/toy_uniform_tensorRT.toml'

# Reading TOML file and converting it into a Python dictionary
with open(toml_file_path, 'r') as toml_file:
    pass_args = toml.load(toml_file)

# Extract the 'passes.tensorrt-calibrate' section and its children
tensorrt_calibrate_config = pass_args.get('passes', {}).get('tensorrt-calibrate', {})
# Extract the 'passes.tensorrt-quantize' section and its children
tensorrt_quantize_config = pass_args.get('passes', {}).get('tensorrt-quantize', {})
# Extract the 'passes.tensorrt-train' section and its children
tensorrt_train_config = pass_args.get('passes', {}).get('tensorrt-train', {})  # Assuming you have a similar section

# Print or return the extracted section
print("TensorRT Quantize Config:", tensorrt_calibrate_config)
print("TensorRT Train Config:", tensorrt_train_config)
print("TensorRT Quantize Config:", tensorrt_quantize_config)

TensorRT Quantize Config: {'by': 'type', 'calibrator': ['percentile', 'max', 'mse', 'entropy'], 'percentiles': [99], 'report': True, 'default': {'config': {'quantize': False}}}
TensorRT Train Config: {}
TensorRT Quantize Config: {'default': {'config': {'name': 'integer', 'data_in_width': 8, 'data_in_frac_width': 4, 'weight_width': 8, 'weight_frac_width': 9, 'bias_width': 8, 'bias_frac_width': 9}}}


In [8]:
task = 'classification'

# get the input generator
input_generator = InputGenerator(
    data_module=data_module,
    model_info=model_info,
    task=task,
    which_dataloader="val",
)

train_generator = InputGenerator(
    model_info=model_info,
    data_module=data_module,
    task=task,
    which_dataloader="train",
)

val_generator = InputGenerator(
    model_info=model_info,
    data_module=data_module,
    task=task,
    which_dataloader="train",
)

pass_argss = [tensorrt_calibrate_config, tensorrt_train_config, tensorrt_quantize_config]
for pass_args in pass_argss:
    pass_args["train_generator"] = train_generator
    pass_args["val_generator"] = val_generator
    pass_args['data_loader'] = data_module.train_dataloader()

# a demonstration of how to feed an input value to the model
dummy_in = next(iter(input_generator))
_ = model(**dummy_in)

mg, _ = init_metadata_analysis_pass(mg, None)
mg, _ = add_common_metadata_analysis_pass(mg, {"dummy_in": dummy_in})
mg, _ = add_software_metadata_analysis_pass(mg, None)

In [9]:
'''  
./ch transform --config configs/examples/toy_uniform_tensorRT.toml --load /root/mase/TensorRTDev/jsc-tiny_classification_jsc_2024-03-03/software/training_ckpts/best.ckpt --load-type pl
'''

# Reload the transform passes
from chop.passes.graph import (
    save_node_meta_param_interface_pass,
    report_node_meta_param_analysis_pass,
    profile_statistics_analysis_pass,
    add_common_metadata_analysis_pass,
    init_metadata_analysis_pass,
    add_software_metadata_analysis_pass,
    tensorrt_quantize_transform_pass,
    tensorrt_calibrate_transform_pass,
    tensorrt_train_transform_pass,
    )

# Calibrate model (passing data samples to the quantizer and deciding the best amax for activations)
mg = tensorrt_calibrate_transform_pass(mg, pass_args=tensorrt_calibrate_config)

# Train on fake quantization
mg = tensorrt_train_transform_pass(mg, pass_args=tensorrt_train_config)

# Convert and store to ONNX and then TensorRT
trt_graph = tensorrt_quantize_transform_pass(mg, pass_args=tensorrt_quantize_config)

# Analysis pass
...

INFO     Succeeded in calibrating the model in PyTorch!
I0304 19:52:20.838594 139852009707328 calibrate.py:103] Succeeded in calibrating the model in PyTorch!


KeyError: 'by'

In [ ]:
i = 0
for name, module in mg.model.named_modules():
    print(i,module)
    i += 1